In [4]:
import pandas as pd
import pkg.dbs.postgres
from pkg import config
from pkg import claude


In [5]:
conf = config.getConfig("./config/config.ini")
pconf = conf['postgres']
psql = pkg.dbs.postgres.PSQL(pconf['host'], int(pconf['port']),pconf['db'],pconf['user'],pconf['pw'])

In [ ]:
print(psql.get_foregin_keys())

{"fk_units": {"child_table": "units", "child_column": "commander", "parent_table": "soldierinformation", "parent_column": "id"}, "fk_pi1": {"child_table": "personalinformation", "child_column": "jobname", "parent_table": "jobcategory", "parent_column": "name"}, "fk_pi2": {"child_table": "personalinformation", "child_column": "certificate", "parent_table": "certificate", "parent_column": "id"}, "fk_si2": {"child_table": "soldierinformation", "child_column": "affiliation", "parent_table": "units", "parent_column": "id"}, "fk_si1": {"child_table": "soldierinformation", "child_column": "specialty", "parent_table": "specialty", "parent_column": "id"}}


In [6]:
print(psql.get_db_schema())

{"jobcategory": [{"name": "id", "type": "text", "max_length": null, "nullable": "NO"}, {"name": "name", "type": "text", "max_length": null, "nullable": "NO"}], "personalinformation": [{"name": "marry", "type": "boolean", "max_length": null, "nullable": "NO"}, {"name": "child", "type": "smallint", "max_length": null, "nullable": "NO"}, {"name": "religion", "type": "USER-DEFINED", "max_length": null, "nullable": "NO"}, {"name": "academicability", "type": "USER-DEFINED", "max_length": null, "nullable": "NO"}, {"name": "parentsfather", "type": "boolean", "max_length": null, "nullable": "NO"}, {"name": "jobyear", "type": "smallint", "max_length": null, "nullable": "YES"}, {"name": "parentsmother", "type": "boolean", "max_length": null, "nullable": "NO"}, {"name": "address4", "type": "text", "max_length": null, "nullable": "NO"}, {"name": "jobname", "type": "text", "max_length": null, "nullable": "YES"}, {"name": "id", "type": "text", "max_length": null, "nullable": "NO"}, {"name": "certific

In [ ]:
units = pd.read_csv('./data/부대.csv')
pe = pd.read_csv('./data/pe.csv',encoding='cp949')
pi = pd.read_csv('./data/인적정보.csv')
cert = pd.read_csv('./data/자격증.csv')
job = pd.read_csv('./data/직업.csv',encoding='cp949')
si = pd.read_csv('./data/si.csv',encoding='cp949')
spec = pd.read_csv('./data/특기.csv')

In [ ]:
units.dtypes

In [ ]:
for idx in range(len(units)):
    data = units.loc[idx]
    psql.update_data(data.ID, data.Name)
    # psql.insert_data('units',data.ID,data.Commander,data.SubordinateUnit.split(','),data.Organization
    #                 ,data.Num, data.Name)

In [ ]:
spec.dtypes

In [ ]:
for idx in range(len(spec)):
    data = spec.loc[idx]
    psql.insert_data('specialty',data.ID,data.ClassName,data.ClassCode,
                     data.SpecialtyName)

In [ ]:
cert.dtypes

In [ ]:
for idx in range(len(cert)):
    data = cert.loc[idx]
    psql.insert_data('certificate',data.ID,data.Name)

In [ ]:
job.dtypes

In [ ]:
for idx in range(len(job)):
    data = job.loc[idx]
    psql.insert_data('jobcategory',data.ID,data.Name)

In [ ]:
si.dtypes

In [ ]:
si = si.fillna('')

In [ ]:
for idx in range(len(si)):
    data = si.loc[idx]
    psql.insert_data('soldierinformation',data.ID,data.EnDate,data.DurationDay,data.Rank,data.Specialty,data.Affiliation,data.Discipline.split(','))

In [ ]:
pi.dtypes

In [ ]:
rel = {'기독교', '불교', '천주교', '원불교', '무교', '기타'}
for i, val in enumerate(rel):
    pi.Religion = pi.Religion.replace(to_replace=i+1,value=val)

In [ ]:
aa = {'중', '고', '학사', '석사', '박사'}
for i, val in enumerate(aa):
    pi.AcademicAbility = pi.AcademicAbility.replace(to_replace=i+1,value=val)

In [ ]:
pi.JobYear = pi.JobYear.fillna(0)
pi = pi.fillna('')

In [ ]:
pi.isna().sum()

In [ ]:
for idx in range(len(pi)):
    data = pi.loc[idx]
    psql.insert_data('personalinformation',data.ID,data.Name,data.ResidentIDB,data.ResidentIDF,data.Address1,data.Address2,data.Address3
                     ,data.Address4,data.ParentsFather,data.ParentsMother,data.Marry,data.Child,data.Religion,data.AcademicAbility,data.JobName,data.JobYear,data.Certificate)

In [ ]:
pe.dtypes

In [ ]:
pe.ExamDate = pe.ExamDate.astype(str)

In [ ]:
pe.ExamDate = pe.ExamDate.apply(lambda x: '0' + x if len(x)<=5 else x)

In [ ]:
pe.ExamDate = pe.ExamDate.apply(lambda x: x[:2]+x[2:4]+'28' if x[2:]=='0229' or '0230' else x)

In [ ]:
pe.ExamDate = pe.ExamDate.apply(lambda x : '20'+x[:2]+'-'+x[2:4]+'-'+x[-2:])

In [ ]:
pe

In [ ]:
for idx in range(len(pe)):
    data = pe.loc[idx]
    # print(data)
    psql.insert_data('pe',data.ID,data.BloodType,data.ExamDate,data.ExamLocation,data.Height,data.Chest,data.Weight,data.BpHigh,data.BpLow,data.VisionL,data.VisionR,data.PhyGrade)